In [9]:
import pandas as pd

dataset = pd.read_json('./datasets/DailyNews_300.json')
summary = dataset.iloc[0,2]
text = dataset.iloc[0,3]

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = True)
model = BertForMaskedLM.from_pretrained('bert-base-uncased')


c:\Users\Liora\anaconda3\envs\collabfiltering\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Liora\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationshi

In [20]:
acc = BLANC_help(summary, text, model)
print(acc)

KeyError: 'Invalid key. Only three types of key are available: (1) string, (2) integers for backend Encoding, and (3) slices for data subsetting.'

In [8]:
# The models the authors used:
try:
    from transformers import BertForMaskedLM, BertTokenizer, AdamW, get_linear_schedule_with_warmup
    from transformers import AlbertForMaskedLM, AlbertTokenizer
except ModuleNotFoundError:
    %pip install transformers
    from transformers import BertForMaskedLM, BertTokenizer, AdamW, get_linear_schedule_with_warmup
    from transformers import AlbertForMaskedLM, AlbertTokenizer


In [19]:
def BLANC_help(summary, text, model, M=6, L_min=4, sep=''):
    """
    Calculate BLANC similarity between summary and text using a specified model.

    Parameters:
    - summary (str): The summary text.
    - text (List[List[str]]): List of sentences represented as a list of words.
    - model: BERT model type
    - M (int): Parameter M for the algorithm (default is 6).
    - L_min (int): Minimum length requirement for masked words (default is 4).
    - sep (str): Separator between the inference help (filler/summary) and a sentence from the text (default is '').

    Returns:
    - float: BLANC similarity score.
    """

    filler = '.' * len(summary)
    S = [[0, 0], [0, 0]]

    for sentence in text:
        for i in range(1, M + 1):
            masked_sentence = ' '.join(['<MASK>' if (j - i) % M == 0 and len(sentence[j]) >= L_min else sentence[j] 
                        for j in range(len(sentence))])
            input_base = filler + sep + masked_sentence
            input_help = summary + sep + masked_sentence
            tensor_input = tokenizer(input_base, return_tensors='pt', max_length=600, truncation=True)
            out_base = model(tensor_input)
            out_help = model(input_help)
            masked_tokens = [idx for idx, word in enumerate(masked_sentence) if word == '<MASK>']

            for j in masked_tokens:
                k = int(out_base[j] == sentence[j])
                m = int(out_help[j] == sentence[j])
                S[k][m] += 1

    return (S[0][1] - S[1][0]) / (S[0][0] + S[1][1] + S[0][1] + S[1][0])



Ideas for improvement:
1. add error handling to ensure that the input to the function (summary, text, model) are of the expected types or shapes